In [2]:
import json

import petl as etl

from utils.paths import PUBLISHED, SITE

In [3]:
TARGET = SITE / 'themes/sustainability/_data/calculations/'

TARGET.mkdir(exist_ok=True, parents=True)

In [4]:
data = (
    etl
    .fromcsv(PUBLISHED / 'sustainability/calculations.csv')
    .convert(['date_from', 'date_to'], etl.dateparser('%Y-%m-%d'))
    .convertnumbers()
)

In [ ]:
data

In [6]:
by_impact_type = (
    data
    .aggregate(['impact_type'], sum, 'calculation_tco2e')
    .sort('value', reverse=True)
)

In [ ]:
etl.cat(
    by_impact_type,
)

In [22]:
total_tco2e = list(by_impact_type.aggregate(None, sum, 'value').values('value'))[0]

In [23]:
project_related_emissions = (
    data
    .cut('project_id', 'calculation_tco2e')
    .convert('project_id', lambda f: True if f else False)
    .aggregate(
        'project_id', sum, 'calculation_tco2e'
    )
    .convert('value', lambda f: round(100 * f / total_tco2e, 1))
    .selecttrue('project_id')
    .values('value')[0]
)

In [ ]:

with open(TARGET / 'summary.json', 'w') as f:
    json.dump(
        {
            'total_tCO2e': round(total_tco2e, 1),
            'project_related_emissions': project_related_emissions,
        },
        f
    )

In [9]:
with open(TARGET / 'by_impact.json', 'w') as f:
    json.dump(
        [{ r.impact_type: r.value for r in by_impact_type.convert('value', lambda r: round(100 * r / total_tco2e, 1)).records() }],
        f
    )